In [4]:
import numpy as np
import fasttext
from collections import defaultdict, Counter
import pickle
from pycountry import languages
from lifelong_dnn import LifeLongDNN

In [5]:
def parse_language_data(file_name):
    labels, sentences = [],[]
    with open(file_name) as handle:
        for line in handle.readlines():
            ll = line.split()
            labels.append(ll[0])
            sentences.append(" ".join(ll[1:]))
                             
    print( len(labels), len(sentences))
    return labels, sentences

fast_text_emb = fasttext.load_model('/home/weiwya/lifelong-learning/language_detection/data/langdetect_unsuperised_wordonly.bin')
data_file = '/home/weiwya/lifelong-learning/language_detection/data/all.txt'

data_labels, data_sentences = parse_language_data(data_file)
allLang = Counter(data_labels)
print (len(allLang))

wanted_language = set(['eng', 'cmn', 'jpn', 'spa', 'fra',
                       'deu', 'swe', 'nob', 'dan', 'nld', 
                       'heb', 'ara', 'rus', 'ukr', 'pol',
                       'kor', 'hrv', 'srp', 'vie', 'bos',
                       'fin', 'est', 'cat', 'hun', 'bre', 
                       'hin', 'yue', 'wuu', 'ita', 'por'])

#break into random batch of 3
data_dict = defaultdict(list)
for l, s in zip(data_labels, data_sentences):
    l = l.split('__')[-1]
    if l in wanted_language:
        data_dict[l].append(fast_text_emb.get_sentence_vector(s))

for k, v in data_dict.items():
    k = languages.get(alpha_3 =k).name
    print(k, "\t\t", len(v))

print (len(wanted_language ))

8194317 8194317
350
English 		 1306723
Russian 		 748409
Hungarian 		 276411
Finnish 		 109613
Italian 		 745613
Hebrew 		 195345
Serbian 		 30551
French 		 407854
Danish 		 45022
Spanish 		 315925
German 		 494870
Portuguese 		 355878
Mandarin Chinese 		 61499
Dutch 		 109757
Ukrainian 		 156266
Polish 		 101866
Arabic 		 33887
Korean 		 7202
Catalan 		 6061
Vietnamese 		 10435
Japanese 		 189567
Hindi 		 12184
Swedish 		 35193
Croatian 		 5203
Norwegian Bokmål 		 13619
Wu Chinese 		 4326
Breton 		 7184
Estonian 		 3048
Yue Chinese 		 5925
Bosnian 		 546
30


In [6]:
label_to_idx = {l:i for i, l in enumerate(wanted_language)}
idx_to_label = {label_to_idx[l]:l for l in wanted_language}

# #org by closes language
classes = np.array([
    
    [label_to_idx['swe'], label_to_idx['nob'], label_to_idx['dan']  ],
    [label_to_idx['cmn'], label_to_idx['yue'], label_to_idx['wuu']  ],
    [label_to_idx['rus'], label_to_idx['ukr'], label_to_idx['pol']  ],   
    [label_to_idx['spa'], label_to_idx['ita'], label_to_idx['por']  ],
    [label_to_idx['fin'], label_to_idx['hun'], label_to_idx['est']  ],
    [label_to_idx['eng'], label_to_idx['nld'], label_to_idx['deu']  ],    
    [label_to_idx['hrv'], label_to_idx['srp'], label_to_idx['bos']  ],
    [label_to_idx['jpn'], label_to_idx['kor'], label_to_idx['vie']  ],
    [label_to_idx['heb'], label_to_idx['ara'], label_to_idx['hin']  ],  
    [label_to_idx['fra'], label_to_idx['cat'], label_to_idx['bre']  ],  
])

print(len(classes))

10


In [7]:
def generate_train_test(class_labels, train_size=1000, test_size=2000):
    trains, tests = [], []
    train_labels, test_labels = [], []
    print([idx_to_label[x] for x in class_labels])
    for l in class_labels:
        dd = data_dict[idx_to_label[l]]        
        X_train = dd[:train_size]
        Y_train = [l] * len(X_train)
        trains.append(X_train)
        train_labels += Y_train
        
        
        X_test = dd[train_size: train_size + test_size]
        Y_test = [l]* len(X_test)
        tests.append(X_test)
        test_labels += Y_test
      
    #shuffle orders for good meature 
    idx = np.random.choice(len(train_labels), len(train_labels), replace= False)
    trains = np.vstack(trains)[idx]
    train_labels = np.array(train_labels)[idx]
    
    idx = np.random.choice(len(test_labels), len(test_labels), replace=False)
    tests = np.vstack(tests)[idx]
    test_labels = np.array(test_labels)[idx]
    return trains, train_labels, tests, test_labels

def get_score(prediction, labels):
    acc = (labels == prediction).sum()/labels.shape[0]
    return acc

In [8]:
def run_iter(seed, classes, train_size=250, test_size=2500):


    llf = LifeLongDNN(acorn=seed, parallel=False)
    prv_test, prv_labels = {},{}

    forward_acc = {}
    reverse_eff= defaultdict(list)
    forward_eff= {}
    
    for iteration, cc in enumerate(classes):
        print(iteration)
        train, train_labels , test, test_labels = generate_train_test(cc, train_size=train_size, test_size=test_size)        
        prv_test[iteration] = test
        prv_labels[iteration] = test_labels
        llf.new_forest(train, train_labels)    
        
        p0 = llf.predict(test, representation=iteration, decider=iteration)  
        p1 = llf.predict(test, representation='all', decider=iteration)
        a0 = get_score(p0, test_labels)
        a1 = get_score(p1, test_labels)
        forward_acc[iteration] = a1
        forward_eff[iteration] = (1-a0) / (1-a1)
        
        for j in range(iteration):
            p0 = llf.predict(prv_test[j], representation=j, decider=j)  
            p1 = llf.predict(prv_test[j], representation='all', decider=j)
            e0 = 1 - get_score(p0, prv_labels[j])
            e1 = 1 - get_score(p1, prv_labels[j])
            eff = e0 / e1
            print ('%i, org_error: %s tran_error: %s tran_eff: %s' %(j, e0, e1, eff ))
            reverse_eff[j].append(eff)
        print()
    return seed, (forward_acc, forward_eff, reverse_eff)


In [10]:
res = []
for i in range(10):
    res.append(run_iter(9876-i, classes))
    

0
['swe', 'nob', 'dan']

1
['cmn', 'yue', 'wuu']
0, org_error: 0.30613333333333337 tran_error: 0.2990666666666667 tran_eff: 1.0236290682122158

2
['rus', 'ukr', 'pol']
0, org_error: 0.30613333333333337 tran_error: 0.2996 tran_eff: 1.0218068535825546
1, org_error: 0.6637333333333333 tran_error: 0.6586666666666667 tran_eff: 1.0076923076923074

3
['spa', 'ita', 'por']
0, org_error: 0.30613333333333337 tran_error: 0.29733333333333334 tran_eff: 1.029596412556054
1, org_error: 0.6637333333333333 tran_error: 0.6573333333333333 tran_eff: 1.00973630831643
2, org_error: 0.008933333333333349 tran_error: 0.009199999999999986 tran_eff: 0.9710144927536264

4
['fin', 'hun', 'est']
0, org_error: 0.30613333333333337 tran_error: 0.3002666666666667 tran_eff: 1.019538188277087
1, org_error: 0.6637333333333333 tran_error: 0.6496 tran_eff: 1.0217569786535303
2, org_error: 0.008933333333333349 tran_error: 0.007600000000000051 tran_eff: 1.1754385964912222
3, org_error: 0.0070666666666666655 tran_error: 0.0033

3, org_error: 0.0043999999999999595 tran_error: 0.0031999999999999806 tran_eff: 1.3749999999999956
4, org_error: 0.036933333333333374 tran_error: 0.03973333333333329 tran_eff: 0.9295302013422839
5, org_error: 0.0033333333333332993 tran_error: 0.0037333333333333663 tran_eff: 0.8928571428571259
6, org_error: 0.5128398791540785 tran_error: 0.5183157099697886 tran_eff: 0.9894353369763202
7, org_error: 0.04133333333333333 tran_error: 0.027200000000000002 tran_eff: 1.5196078431372548
8, org_error: 0.00039999999999995595 tran_error: 0.0005333333333333856 tran_eff: 0.7499999999998439

0
['swe', 'nob', 'dan']

1
['cmn', 'yue', 'wuu']
0, org_error: 0.30400000000000005 tran_error: 0.3038666666666666 tran_eff: 1.000438788942519

2
['rus', 'ukr', 'pol']
0, org_error: 0.30400000000000005 tran_error: 0.30146666666666666 tran_eff: 1.008403361344538
1, org_error: 0.6544 tran_error: 0.6549333333333334 tran_eff: 0.9991856677524429

3
['spa', 'ita', 'por']
0, org_error: 0.30400000000000005 tran_error: 0.2

4, org_error: 0.036800000000000055 tran_error: 0.040000000000000036 tran_eff: 0.9200000000000006
5, org_error: 0.0025333333333333874 tran_error: 0.003466666666666618 tran_eff: 0.7307692307692566
6, org_error: 0.5166163141993958 tran_error: 0.5243580060422961 tran_eff: 0.9852358660424919
7, org_error: 0.04173333333333329 tran_error: 0.02839999999999998 tran_eff: 1.4694835680751168

9
['fra', 'cat', 'bre']
0, org_error: 0.30946666666666667 tran_error: 0.29879999999999995 tran_eff: 1.0356983489513611
1, org_error: 0.6557333333333333 tran_error: 0.6526666666666667 tran_eff: 1.0046986721144022
2, org_error: 0.008933333333333349 tran_error: 0.006399999999999961 tran_eff: 1.3958333333333441
3, org_error: 0.00653333333333328 tran_error: 0.003066666666666662 tran_eff: 2.1304347826086816
4, org_error: 0.036800000000000055 tran_error: 0.04039999999999999 tran_eff: 0.9108910891089125
5, org_error: 0.0025333333333333874 tran_error: 0.004133333333333322 tran_eff: 0.6129032258064664
6, org_error: 0.5

4, org_error: 0.036800000000000055 tran_error: 0.038799999999999946 tran_eff: 0.9484536082474254
5, org_error: 0.0028000000000000247 tran_error: 0.0037333333333333663 tran_eff: 0.75
6, org_error: 0.523036253776435 tran_error: 0.5253021148036254 tran_eff: 0.9956865564342199

8
['heb', 'ara', 'hin']
0, org_error: 0.30746666666666667 tran_error: 0.3009333333333334 tran_eff: 1.0217102348249887
1, org_error: 0.6619999999999999 tran_error: 0.6562666666666667 tran_eff: 1.0087362860625761
2, org_error: 0.008266666666666644 tran_error: 0.006399999999999961 tran_eff: 1.291666666666671
3, org_error: 0.004533333333333278 tran_error: 0.002666666666666706 tran_eff: 1.6999999999999542
4, org_error: 0.036800000000000055 tran_error: 0.038799999999999946 tran_eff: 0.9484536082474254
5, org_error: 0.0028000000000000247 tran_error: 0.0033333333333332993 tran_eff: 0.840000000000016
6, org_error: 0.523036253776435 tran_error: 0.5288897280966767 tran_eff: 0.9889325240985364
7, org_error: 0.037866666666666715

4, org_error: 0.03626666666666667 tran_error: 0.03973333333333329 tran_eff: 0.9127516778523501
5, org_error: 0.003066666666666662 tran_error: 0.0037333333333333663 tran_eff: 0.821428571428563

7
['jpn', 'kor', 'vie']
0, org_error: 0.3044 tran_error: 0.29479999999999995 tran_eff: 1.0325644504748985
1, org_error: 0.6476 tran_error: 0.6568 tran_eff: 0.9859926918392203
2, org_error: 0.008000000000000007 tran_error: 0.006800000000000028 tran_eff: 1.1764705882352904
3, org_error: 0.006800000000000028 tran_error: 0.0028000000000000247 tran_eff: 2.4285714285714173
4, org_error: 0.03626666666666667 tran_error: 0.03986666666666672 tran_eff: 0.9096989966555173
5, org_error: 0.003066666666666662 tran_error: 0.0037333333333333663 tran_eff: 0.821428571428563
6, org_error: 0.5243580060422961 tran_error: 0.520392749244713 tran_eff: 1.0076197387518142

8
['heb', 'ara', 'hin']
0, org_error: 0.3044 tran_error: 0.2945333333333333 tran_eff: 1.0334993209597103
1, org_error: 0.6476 tran_error: 0.653333333333

4, org_error: 0.03759999999999997 tran_error: 0.03813333333333335 tran_eff: 0.9860139860139846

6
['hrv', 'srp', 'bos']
0, org_error: 0.3024 tran_error: 0.29666666666666663 tran_eff: 1.0193258426966294
1, org_error: 0.6498666666666666 tran_error: 0.6548 tran_eff: 0.9924658928935042
2, org_error: 0.008266666666666644 tran_error: 0.00666666666666671 tran_eff: 1.2399999999999887
3, org_error: 0.006000000000000005 tran_error: 0.003066666666666662 tran_eff: 1.9565217391304395
4, org_error: 0.03759999999999997 tran_error: 0.03920000000000001 tran_eff: 0.9591836734693866
5, org_error: 0.0029333333333333433 tran_error: 0.0036000000000000476 tran_eff: 0.8148148148148068

7
['jpn', 'kor', 'vie']
0, org_error: 0.3024 tran_error: 0.29546666666666666 tran_eff: 1.0234657039711192
1, org_error: 0.6498666666666666 tran_error: 0.6526666666666667 tran_eff: 0.9957099080694585
2, org_error: 0.008266666666666644 tran_error: 0.006266666666666643 tran_eff: 1.3191489361702142
3, org_error: 0.00600000000000000